# Deep Q-Network with Tensorflow

Implementing a DQN using tensorflow and solving the lunar lander

### 1. Import the required packages

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

import gym
import random
import numpy as np
from collections import namedtuple, deque
import matplotlib.pyplot as plt
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [7]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


### Define the Model

In [63]:

class QNetwork():

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        self.model = Sequential([
            Dense(units=fc1_units, input_shape=(state_size,), activation='relu'),
            Dense(units=fc2_units, activation='relu'),
            Dense(units=action_size, activation='relu'),
        ])
    
        self.model.compile(optimizer=Adam(learning_rate=0.0001), loss="mean_squared_error")
        
        self.action_size = action_size

    def forward(self, state):

        return self.model.predict(x=np.array([state]))

    def learn(self, samples, labels, batch_size=32, epochs=100, verbose=2):
        self.model.fit(x=samples, y=labels, epochs=epochs, verbose=0)
        
    def save(self, location):
        tf.keras.models.save_model(self.model, location)
        


### Define the Agent

In [66]:
BUFFER_SIZE = 1000  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 0.001              # for soft update of target parameters
LR = 0.0005               # learning rate 
UPDATE_EVERY = 500        # how often to update the network

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed)
        self.qnetwork_target = QNetwork(state_size, action_size, seed)
        
        self.copy_network(self.qnetwork_local , self.qnetwork_target)
        
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done, skip_learn=False):

        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) >= BUFFER_SIZE and not skip_learn:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        action_values = self.qnetwork_local.forward(state)

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values)
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = []
        i =0
        for state in next_states:
            output = self.qnetwork_target.forward(state)[0]
            output[actions[i]] =rewards[i]
            Q_targets_next.append(output)
            i +=1
        # Compute Q targets for current states 
        Q_targets = [] 
        
        i =0
        for reward in rewards:
            reward + (gamma * Q_targets_next[i] * (1 - dones[i]))
            i +=1
            Q_targets.append(reward)
        
        Q_targets = np.array(Q_targets)
        


        self.qnetwork_local.learn(states,Q_targets)

        # ------------------- update target network ------------------- #
        self.soft_update_target(TAU)                     

    def copy_network(self, local_model, target_model):
        target_model_update = tf.keras.models.clone_model(local_model.model)
        target_model.model = target_model_update
        
    def soft_update_target(self, TAU):
        target_weights = np.array(self.qnetwork_target.model.get_weights())
        local_weights = np.array(self.qnetwork_local.model.get_weights())
        self.qnetwork_target.model.set_weights(target_weights + TAU*(local_weights-target_weights))

    
    def save(self):
        self.qnetwork_target.save("checkpoint.target.ckpt")
        self.qnetwork_local.save("checkpoint.local.ckpt")
    


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = np.vstack([e.state for e in experiences if e is not None])
        actions = np.vstack([e.action for e in experiences if e is not None])
        rewards = np.vstack([e.reward for e in experiences if e is not None])
        next_states = np.vstack([e.next_state for e in experiences if e is not None])
        dones = np.vstack([e.done for e in experiences if e is not None])
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

## Watch an untrained agent

In [ ]:
agent = Agent(state_size=8, action_size=4, seed=0)

# watch an untrained agent
state = env.reset()
for j in range(200):
    action = agent.act(state)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 
        
env.close()

### Lets Train an agent

In [ ]:
agent = Agent(state_size=8, action_size=4, seed=0)

def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    
    # We need to start with some random games
    random_game_count = 0
    state = env.reset()
    while len(agent.memory) < BUFFER_SIZE:
        random_game_count += 1
        state = env.reset()
        score = 0
        done = False
        step_num = 1
        while not done:

            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done, skip_learn=True)
            state = next_state
            score += reward
            
        if random_game_count%10 == 0:
            print("Random Game Count: "+ str(random_game_count))
            print("Memory Size: "+ str(len(agent.memory)))
            
    print("Done Filling Up memory")
 
    
    
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        print(' Epsilon: '+str(eps))
        if i_episode % 10 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            agent.save()
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Random Game Count: 10
Memory Size: 649
Done Filling Up memory
Episode 1	Average Score: -319.33 Epsilon: 0.995
Episode 2	Average Score: -192.40 Epsilon: 0.990025
Episode 3	Average Score: -197.77 Epsilon: 0.985074875
Episode 4	Average Score: -209.69 Epsilon: 0.9801495006250001
Episode 5	Average Score: -226.04 Epsilon: 0.9752487531218751
Episode 6	Average Score: -240.40 Epsilon: 0.9703725093562657
Episode 7	Average Score: -234.22 Epsilon: 0.9655206468094844
Episode 8	Average Score: -215.09 Epsilon: 0.960693043575437
Episode 9	Average Score: -201.18 Epsilon: 0.9558895783575597
Episode 10	Average Score: -191.18 Epsilon: 0.9511101304657719
Episode 10	Average Score: -191.18
INFO:tensorflow:Assets written to: checkpoint.target.ckpt/assets
INFO:tensorflow:Assets written to: checkpoint.local.ckpt/assets
Episode 11	Average Score: -187.69 Epsilon: 0.946354579813443
Episode 12	Average Score: -181.87 Epsilon: 0.9416228069143757
Episode 13	Average Score: -185.28 Epsilon: 0.9369146928798039
Episode 14

Episode 107	Average Score: -150.73 Epsilon: 0.5848838636585911
Episode 108	Average Score: -152.11 Epsilon: 0.5819594443402982
Episode 109	Average Score: -152.70 Epsilon: 0.5790496471185967
Episode 110	Average Score: -153.34 Epsilon: 0.5761543988830038
Episode 110	Average Score: -153.34
INFO:tensorflow:Assets written to: checkpoint.target.ckpt/assets
INFO:tensorflow:Assets written to: checkpoint.local.ckpt/assets
Episode 111	Average Score: -152.93 Epsilon: 0.5732736268885887
Episode 112	Average Score: -153.52 Epsilon: 0.5704072587541458
Episode 113	Average Score: -153.02 Epsilon: 0.567555222460375
Episode 114	Average Score: -152.96 Epsilon: 0.5647174463480732
Episode 115	Average Score: -152.03 Epsilon: 0.5618938591163328
Episode 116	Average Score: -151.58 Epsilon: 0.5590843898207511
Episode 117	Average Score: -149.03 Epsilon: 0.5562889678716474
Episode 118	Average Score: -148.44 Epsilon: 0.5535075230322891
Episode 119	Average Score: -150.25 Epsilon: 0.5507399854171277
Episode 120	Averag

INFO:tensorflow:Assets written to: checkpoint.target.ckpt/assets
INFO:tensorflow:Assets written to: checkpoint.local.ckpt/assets
Episode 211	Average Score: -140.90 Epsilon: 0.3472722151889232
Episode 212	Average Score: -140.33 Epsilon: 0.3455358541129786
Episode 213	Average Score: -140.72 Epsilon: 0.3438081748424137
Episode 214	Average Score: -140.42 Epsilon: 0.3420891339682016
Episode 215	Average Score: -141.43 Epsilon: 0.3403786882983606
Episode 216	Average Score: -140.11 Epsilon: 0.3386767948568688
Episode 217	Average Score: -140.78 Epsilon: 0.33698341088258443
Episode 218	Average Score: -141.74 Epsilon: 0.3352984938281715
Episode 219	Average Score: -140.36 Epsilon: 0.33362200135903064
Episode 220	Average Score: -140.30 Epsilon: 0.33195389135223546
Episode 220	Average Score: -140.30
INFO:tensorflow:Assets written to: checkpoint.target.ckpt/assets
INFO:tensorflow:Assets written to: checkpoint.local.ckpt/assets
Episode 221	Average Score: -139.90 Epsilon: 0.3302941218954743
Episode 222

Episode 313	Average Score: -142.47 Epsilon: 0.20826882814336947
Episode 314	Average Score: -143.15 Epsilon: 0.20722748400265262
Episode 315	Average Score: -142.46 Epsilon: 0.20619134658263935
Episode 316	Average Score: -142.88 Epsilon: 0.20516038984972615
Episode 317	Average Score: -141.58 Epsilon: 0.2041345879004775
Episode 318	Average Score: -140.76 Epsilon: 0.2031139149609751
Episode 319	Average Score: -140.97 Epsilon: 0.20209834538617025
Episode 320	Average Score: -139.84 Epsilon: 0.2010878536592394
Episode 320	Average Score: -139.84
INFO:tensorflow:Assets written to: checkpoint.target.ckpt/assets
INFO:tensorflow:Assets written to: checkpoint.local.ckpt/assets
Episode 321	Average Score: -140.94 Epsilon: 0.2000824143909432
Episode 322	Average Score: -141.12 Epsilon: 0.19908200231898848
Episode 323	Average Score: -141.08 Epsilon: 0.19808659230739353
Episode 324	Average Score: -143.50 Epsilon: 0.19709615934585656
Episode 325	Average Score: -142.62 Epsilon: 0.19611067854912728
Episode 